In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Equação da Onda 1D - Ondas na Corda
\begin{equation}
\frac{\partial^2 u}{\partial t^2} =
c^2 \frac{\partial^2 u}{\partial x^2}, \quad x\in (0,L),\ t\in (0,T]
\tag{1}
\end{equation}


\begin{split}
u(x,0) &= I(x), \quad x\in [0,L]
\\
\frac{\partial}{\partial t}u(x,0) &= 0, \quad x\in [0,L]
\\
u(0,t) & = 0, \quad  t\in (0,T],
\\
u(L,t) & = 0, \quad  t\in (0,T]
\thinspace .\end{split}

## Discretização
$$x_i = i\Delta x,\quad i=0,\ldots,N_x,$$
$$t_i = n\Delta t,\quad n=0,\ldots,N$$

O domínio discretizado $(0,L)\times (0,T]$ pode se expresso pela malha:

$$\frac{\partial^2}{\partial t^2} u(x_i, t_n) =
     c^2\frac{\partial^2}{\partial x^2} u(x_i, t_n),$$


Aproximamos a derivada temporal por diferenças centradas:

$$\frac{\partial^2}{\partial t^2}u(x_i,t_n)\approx
\frac{u_i^{n+1} - 2u_i^n + u^{n-1}_i}{\Delta t^2}$$

A derivada espacial também é aproximada por diferenças centradas:

$$\frac{\partial^2}{\partial x^2}u(x_i,t_n)\approx
\frac{u_{i+1}^{n} - 2u_i^n + u^{n}_{i-1}}{\Delta x^2}$$

Substituindo em (1) ficamos com:

\begin{equation}\frac{u_i^{n+1} - 2u_i^n + u^{n-1}_i}{\Delta t^2} =
     c^2\frac{u_{i+1}^{n} - 2u_i^n + u^{n}_{i-1}}{\Delta x^2}
\end{equation}

Obtemos então a fórmula:
\begin{equation}
u^{n+1}_i = -u^{n-1}_i + 2u^n_i + C^2
     \left(u^{n}_{i+1}-2u^{n}_{i} + u^{n}_{i-1}\right)
\end{equation}
onde $C = c\frac{\Delta t}{\Delta x}$ (dimensionless Courant number)





In [ ]:
def solver(I, V, f, c, L, Nx, C, T, user_action=None):
    """Solve u_tt=c^2*u_xx + f on (0,L)x(0,T]."""
    x = np.linspace(0, L, Nx+1)   # mesh points in space
    dx = x[1] - x[0]
    dt = C*dx/c
    N = int(round(T/dt))
    t = np.linspace(0, N*dt, N+1) # mesh points in time
    C2 = C**2                  # help variable in the scheme
    if f is None or f == 0 :
        f = lambda x, t: 0
    if V is None or V == 0:
        V = lambda x: 0

    u   = np.zeros(Nx+1)   # solution array at new time level
    u_1 = np.zeros(Nx+1)   # solution at 1 time level back
    u_2 = np.zeros(Nx+1)   # solution at 2 time levels back

    import time;  t0 = time.clock()  # for measuring CPU time

    # Load initial condition into u_1
    for i in range(0,Nx+1):
        u_1[i] = I(x[i])

    if user_action is not None:
        user_action(u_1, x, t, 0)

    # Special formula for first time step
    n = 0
    for i in range(1, Nx):
        u[i] = u_1[i] + dt*V(x[i]) + \
               0.5*C2*(u_1[i-1] - 2*u_1[i] + u_1[i+1]) + \
               0.5*dt**2*f(x[i], t[n])
    u[0] = 0;  u[Nx] = 0

    if user_action is not None:
        user_action(u, x, t, 1)

    u_2[:], u_1[:] = u_1, u

    for n in range(1, N):
        # Update all inner points at time t[n+1]
        for i in range(1, Nx):
            u[i] = - u_2[i] + 2*u_1[i] + \
                     C2*(u_1[i-1] - 2*u_1[i] + u_1[i+1]) + \
                     dt**2*f(x[i], t[n])

        # Insert boundary conditions
        u[0] = 0;  u[Nx] = 0
        if user_action is not None:
            if user_action(u, x, t, n+1):
                break

        # Switch variables before next step
        u_2[:], u_1[:] = u_1, u

    cpu_time = t0 - time.clock()
    return u, x, t, cpu_time